In [1]:
import pandas as pd
import numpy as np
import pickle
from ast import literal_eval

## Input Data

In [2]:
df = pd.read_csv('../Dataset/clean_data.csv')
df.head(3)

,type,city,hotelFacilities,nearestPointOfInterests,starRating,size,originalRate,baseOccupancy,maxChildOccupancy,maxChildAge,isBreakfastIncluded,isWifiIncluded,isRefundable,hasLivingRoom,roomFacilities
0,Hotel,Badung,"[""CARPARK"",""ELEVATOR"",""HAS_24_HOUR_ROOM_SERVIC...","[{""landmarkId"":""91589773100576"",""geoId"":null,""...",4,46.0,1227273,3,1,5,1,1,1,0,"[""AIR_CONDITIONING"",""BALCONY_TERRACE"",""BATHROB..."
1,Hotel,Badung,"[""CARPARK"",""ELEVATOR"",""HAS_24_HOUR_ROOM_SERVIC...","[{""landmarkId"":""91589773100576"",""geoId"":null,""...",4,31.0,596694,2,1,5,0,1,1,0,"[""AIR_CONDITIONING"",""BATHROBES"",""BATHTUB"",""BLA..."
2,Hotel,Badung,"[""CARPARK"",""ELEVATOR"",""HAS_24_HOUR_ROOM_SERVIC...","[{""landmarkId"":""91589773100576"",""geoId"":null,""...",4,52.0,1450413,2,1,5,1,1,1,1,"[""AIR_CONDITIONING"",""BALCONY_TERRACE"",""BATHROB..."


In [3]:
print(f'Total number of rows: {df.shape[0]}')
print(f'Total number of columns: {df.shape[1]}')

Total number of rows: 5160
Total number of columns: 15


### Feature Processing Function

In [4]:
# Rooms facilities processing
# How to use : Facilities = getFacilities(df['Facilities'].tolist())
def getFacilities(facilitiesList):
    facilities = []
    for i in facilitiesList:
        m = int(i.count('"') / 2)
        for j in range(1, m+1, 2):
            temp = i.split('"')[j]
            if temp not in facilities:
                facilities.append(temp)
    # facilities = [f"{prefix}_" + string for string in facilities]
    return facilities

In [5]:
# seperate the target variable
dfPrice = df[['originalRate']]
dfPrice.head(2)

,originalRate
0,1227273
1,596694


## Getting extraxcted features

### Getting the data

In [6]:
# Hotel Facilities to List
hotelFacilities = getFacilities(df['hotelFacilities'].tolist())
print(f' Total Unique Hotel Facilities: {len(hotelFacilities)}')

 Total Unique Hotel Facilities: 238


In [7]:
# Room Facilities to List
roomFacilities = getFacilities(df['roomFacilities'].tolist())
print(f' Total Unique Hotel Facilities: {len(roomFacilities)}')

 Total Unique Hotel Facilities: 110


In [8]:
# nearestPointOfInterests to List
# landmarkType is present at indexes [19, 49, 79, 109]

landmark = df['nearestPointOfInterests'].to_list()
searchText = 'landmarkType'
nearestPoint = []
loopCount = 0
for x in range (len(landmark)):
    loopCount += 1
    i = 0
    column = landmark[x].split('"')
    matched_indexes = []
    while i < len(column):
        if searchText == column[i]:
            matched_indexes.append(i)
        i += 1
    for y in matched_indexes:
        if column[y+2] not in nearestPoint:
            nearestPoint.append(column[y+2])
if 'OTHERS' in nearestPoint:
    nearestPoint.remove('OTHERS')

print(f' Total Unique Hotel Facilities: {len(nearestPoint)}')

 Total Unique Hotel Facilities: 19


In [9]:
# Create new dataframe
dfHotelEncode = pd.DataFrame(columns=hotelFacilities)
dfRoomEncode = pd.DataFrame(columns=roomFacilities)
dfPointEncode = pd.DataFrame(columns=nearestPoint)

### Extra Features

In [10]:
dfHotelEx = df[['originalRate','type','city','starRating']]
dfRoomEx = df[['originalRate','size', 'baseOccupancy', 'maxChildOccupancy', 'maxChildAge', 'isBreakfastIncluded', 'isWifiIncluded', 'isRefundable', 'hasLivingRoom']]
print(dfHotelEx.shape)
print(dfRoomEx.shape)

(5160, 4)
(5160, 9)


#### Hotel Extra Features

In [11]:
dfHotelEx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5160 entries, 0 to 5159
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   originalRate  5160 non-null   int64 
 1   type          5160 non-null   object
 2   city          5160 non-null   object
 3   starRating    5160 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.4+ KB


In [12]:
dfHotelEx['city'].unique()

array(['Badung', 'Denpasar', 'Gianyar', 'Sanur', 'Bangli', 'Buleleng',
       'Klungkung', 'Tabanan', 'Jembrana', 'Karangasem'], dtype=object)

In [13]:
dfHotelEx['type'].unique()

array(['Hotel', 'Resor', 'Apartemen', 'Vila', 'Guest House', 'Homestay',
       'B&B', 'Hostel', 'Camping', 'Lainnya', 'Hotel Kapsul'],
      dtype=object)

In [14]:
cityEncode = pd.get_dummies(dfHotelEx['city'], prefix='City')
typeEncode = pd.get_dummies(dfHotelEx['type'], prefix='Type')
dfHotelEx = pd.concat([dfHotelEx, cityEncode, typeEncode], axis=1)

In [15]:
dfHotelEx.drop(['city', 'type'], axis=1, inplace=True)
print(dfHotelEx.shape)
dfHotelEx.head(2)

(5160, 23)


,originalRate,starRating,City_Badung,City_Bangli,City_Buleleng,City_Denpasar,City_Gianyar,City_Jembrana,City_Karangasem,City_Klungkung,...,Type_B&B,Type_Camping,Type_Guest House,Type_Homestay,Type_Hostel,Type_Hotel,Type_Hotel Kapsul,Type_Lainnya,Type_Resor,Type_Vila
0,1227273,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,596694,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [16]:
dfHotelEx.describe()

,originalRate,starRating,City_Badung,City_Bangli,City_Buleleng,City_Denpasar,City_Gianyar,City_Jembrana,City_Karangasem,City_Klungkung,...,Type_B&B,Type_Camping,Type_Guest House,Type_Homestay,Type_Hostel,Type_Hotel,Type_Hotel Kapsul,Type_Lainnya,Type_Resor,Type_Vila
count,5.160000e+03,5160.000000,5160.000000,5160.000000,5160.000000,5160.000000,5160.000000,5160.000000,5160.000000,5160.000000,...,5160.000000,5160.000000,5160.000000,5160.000000,5160.000000,5160.000000,5160.000000,5160.000000,5160.000000,5160.000000
mean,1.313368e+06,3.355039,0.431202,0.010659,0.073256,0.047481,0.197481,0.012984,0.052326,0.098256,...,0.020155,0.002713,0.127907,0.053295,0.020930,0.475388,0.000775,0.000969,0.136822,0.145349
std,1.240014e+06,1.162972,0.495292,0.102700,0.260581,0.212685,0.398137,0.113218,0.222704,0.297689,...,0.140544,0.052023,0.334019,0.224642,0.143165,0.499442,0.027834,0.031117,0.343692,0.352486
min,4.031300e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.545450e+05,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.701100e+05,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.714383e+06,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,6.611571e+06,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Room Extra Features

In [17]:
dfRoomEx.head(2)

,originalRate,size,baseOccupancy,maxChildOccupancy,maxChildAge,isBreakfastIncluded,isWifiIncluded,isRefundable,hasLivingRoom
0,1227273,46.0,3,1,5,1,1,1,0
1,596694,31.0,2,1,5,0,1,1,0


### Hotel Feature

In [18]:
dfHotel = pd.concat([df.pop('hotelFacilities'), dfHotelEncode], axis=1)
dfHotel = pd.concat([dfPrice, dfHotel], axis=1)
print(dfHotel.shape)
dfHotel.head(2)

(5160, 240)


,originalRate,hotelFacilities,CARPARK,ELEVATOR,HAS_24_HOUR_ROOM_SERVICE,RESTAURANT,RESTAURANT_FOR_BREAKFAST,RESTAURANT_FOR_DINNER,RESTAURANT_FOR_LUNCH,SAFETY_DEPOSIT_BOX,...,INFINITY_POOL,HEALTH_CLUB,WATER_PARK_ACCESS_SURCHARGE,PRIVATE_BEACH,SURFING,PRIVATE_BEACH_NEARBY,BEACH_SUN_LOUNGERS,DARTS,ENTERTAINMENT_PROGRAMME_FOR_CHILDREN,KARAOKE
0,1227273,"[""CARPARK"",""ELEVATOR"",""HAS_24_HOUR_ROOM_SERVIC...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,596694,"[""CARPARK"",""ELEVATOR"",""HAS_24_HOUR_ROOM_SERVIC...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# add value to decoded columns
for i in range (len(dfHotel)):
    value = dfHotel['hotelFacilities'][i]
    if value == '[]':
        dfHotel.drop([i], axis=0, inplace=True)
        continue
    for j in range (2, len(dfHotel.columns)):
        column_name = dfHotel.columns[j]
        if column_name in value:
            dfHotel.loc[i, column_name] = 1
        else:
            dfHotel.loc[i, column_name] = 0
print(dfHotel.shape)
dfHotel.head()

(5100, 240)


,originalRate,hotelFacilities,CARPARK,ELEVATOR,HAS_24_HOUR_ROOM_SERVICE,RESTAURANT,RESTAURANT_FOR_BREAKFAST,RESTAURANT_FOR_DINNER,RESTAURANT_FOR_LUNCH,SAFETY_DEPOSIT_BOX,...,INFINITY_POOL,HEALTH_CLUB,WATER_PARK_ACCESS_SURCHARGE,PRIVATE_BEACH,SURFING,PRIVATE_BEACH_NEARBY,BEACH_SUN_LOUNGERS,DARTS,ENTERTAINMENT_PROGRAMME_FOR_CHILDREN,KARAOKE
0,1227273,"[""CARPARK"",""ELEVATOR"",""HAS_24_HOUR_ROOM_SERVIC...",1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,596694,"[""CARPARK"",""ELEVATOR"",""HAS_24_HOUR_ROOM_SERVIC...",1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1450413,"[""CARPARK"",""ELEVATOR"",""HAS_24_HOUR_ROOM_SERVIC...",1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,855372,"[""CARPARK"",""ELEVATOR"",""HAS_24_HOUR_ROOM_SERVIC...",1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2545455,"[""CARPARK"",""COFFEE_SHOP"",""ELEVATOR"",""HAS_24_HO...",1,1,1,1,1,1,1,1,...,0,0,0,1,0,0,0,0,0,0


In [20]:
# Remove hotelFacilities column
dfHotel.drop('hotelFacilities', axis=1, inplace=True)

In [21]:
for i in range (1, len(dfHotel.columns)):
    dfHotel = dfHotel.astype({dfHotel.columns[i]: int})

### Room Feature

In [22]:
dfRoom = pd.concat([df.pop('roomFacilities'), dfRoomEncode], axis=1)
dfRoom = pd.concat([dfPrice, dfRoom], axis=1)
print(dfRoom.shape)
dfRoom.head(2)

(5160, 112)


,originalRate,roomFacilities,AIR_CONDITIONING,BALCONY_TERRACE,BATHROBES,BATHTUB,BLACKOUT_DRAPES_CURTAINS,COFFEE_TEA_MAKER,COMPLIMENTARY_BOTTLED_WATER,DESK,...,EXTRA_BEDS_AVAILABLE,FREE_INTERNATIONAL_CALLS,ROLLAWAY_OR_EXTRA_BEDS,YARD,VIDEO_GAMES,IN_ROOM_SAFE_SURCHARGE,HOUSEKEEPING_ON_REQUEST,NO_HOUSEKEEPING,CHANGING_TABLE,MICROWAVE_SURCHARGE
0,1227273,"[""AIR_CONDITIONING"",""BALCONY_TERRACE"",""BATHROB...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,596694,"[""AIR_CONDITIONING"",""BATHROBES"",""BATHTUB"",""BLA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# add value to decoded columns
for i in range(len(dfRoom)):
    value = dfRoom['roomFacilities'][i]
    if value == '[]':
        dfRoom.drop([i], axis=0, inplace=True)
        continue
    for j in range (2, len(dfRoom.columns)):
        column_name = dfRoom.columns[j]
        if column_name in value:
            dfRoom.loc[i, column_name] = 1
        else:
            dfRoom.loc[i, column_name] = 0
print(dfRoom.shape)


(4934, 112)


In [24]:
dfRoom.drop('roomFacilities', axis=1, inplace=True)

In [25]:
for i in range (1, len(dfRoom.columns)):
    dfRoom = dfRoom.astype({dfRoom.columns[i]: int})

### Point of Interest Feature

In [26]:
dfPoint = pd.concat([df.pop('nearestPointOfInterests'), dfPointEncode], axis=1)
dfPoint = pd.concat([dfPrice, dfPoint], axis=1)
dfPoint.head(2)

,originalRate,nearestPointOfInterests,SHOPPING_AREA,OFFICIAL_BUILDING,RESTAURANT,ATTRACTION,BEACH,MONUMENT,TERMINAL,PARK,...,MUSEUM,GALLERY,PLACE_OF_WORSHIP,TRAIN_STATION,ZOO,ENTERTAINMENT,GARDEN,THEATER,STORE,SCHOOL
0,1227273,"[{""landmarkId"":""91589773100576"",""geoId"":null,""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,596694,"[{""landmarkId"":""91589773100576"",""geoId"":null,""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# add value to encoded columns
for i in range (len(dfPoint)):
    value = dfPoint['nearestPointOfInterests'][i]
    if value == '[]':
        dfPoint.drop([i], axis=0, inplace=True)
        continue
    for j in range (2, len(dfPoint.columns)):
        column_name = dfPoint.columns[j]
        if column_name in value:
            dfPoint.loc[i, column_name] = 1
        else:
            dfPoint.loc[i, column_name] = 0
print(dfPoint.shape)
dfPoint.head()

(5144, 21)


,originalRate,nearestPointOfInterests,SHOPPING_AREA,OFFICIAL_BUILDING,RESTAURANT,ATTRACTION,BEACH,MONUMENT,TERMINAL,PARK,...,MUSEUM,GALLERY,PLACE_OF_WORSHIP,TRAIN_STATION,ZOO,ENTERTAINMENT,GARDEN,THEATER,STORE,SCHOOL
0,1227273,"[{""landmarkId"":""91589773100576"",""geoId"":null,""...",1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,596694,"[{""landmarkId"":""91589773100576"",""geoId"":null,""...",1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1450413,"[{""landmarkId"":""91589773100576"",""geoId"":null,""...",1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,855372,"[{""landmarkId"":""91589773100576"",""geoId"":null,""...",1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2545455,"[{""landmarkId"":""91589773100576"",""geoId"":null,""...",1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
dfPoint.drop('nearestPointOfInterests', axis=1, inplace=True)

In [29]:
# Convert to int
for i in range (1, len(dfPoint.columns)):
    dfPoint = dfPoint.astype({dfPoint.columns[i]: int})

### Add Prefix

In [30]:
dfHotel = dfHotel.add_prefix('Hotel_')
dfRoom = dfRoom.add_prefix('Room_')
dfPoint = dfPoint.add_prefix('Point_')

## Feature Selection

In [31]:
def featureSelection(df):
    # Remove features with below 0.1 or above 0.9 mean
    total1 = 0
    total2 = 0
    deleteIndex = []
    for i in range (1, len(df.columns)):
        if df[df.columns[i]].mean() < 0.1:
            total1 +=1
            deleteIndex.append(df.columns[i])
        if df[df.columns[i]].mean() > 0.9:
            total2 +=1
            deleteIndex.append(df.columns[i])
    print(f'Total Columns (Before Selection) : {len(df.columns)}')

    df.drop(deleteIndex, axis=1, inplace=True)
    return { 'total01': total1, 'total09': total2, 'deleteIndex': deleteIndex, 'dfCol': df.columns, 'df': df}

In [32]:
roomSelection = featureSelection(dfRoom)
print(f'Total Columns with mean < 0.1 : {roomSelection["total01"]}')
print(f'Total Columns with mean > 0.9 : {roomSelection["total09"]}')
print(f'Total deleted index : {len(roomSelection["deleteIndex"])}')
print(f'Total Columns Remaining : {len(roomSelection["dfCol"])}')

Total Columns (Before Selection) : 111
Total Columns with mean < 0.1 : 75
Total Columns with mean > 0.9 : 2
Total deleted index : 77
Total Columns Remaining : 34


In [33]:
hotelSelection = featureSelection(dfHotel)
print(f'Total Columns with mean < 0.1 : {hotelSelection["total01"]}')
print(f'Total Columns with mean > 0.9 : {hotelSelection["total09"]}')
print(f'Total deleted index : {len(hotelSelection["deleteIndex"])}')
print(f'Total Columns Remaining : {len(hotelSelection["dfCol"])}')

Total Columns (Before Selection) : 239
Total Columns with mean < 0.1 : 122
Total Columns with mean > 0.9 : 0
Total deleted index : 122
Total Columns Remaining : 117


In [35]:
hotelFacilities = hotelSelection['df'].columns
roomFacilities = roomSelection['df'].columns
nearestPointOfInterests = dfPoint.columns

In [36]:
# export hotelFacilities, roomFacilities, nearestPointOfInterests list to another notebook
with open('../Notebook/Variable/hotelFacilities.pkl', 'wb') as f:
    pickle.dump(hotelFacilities, f)
with open('../Notebook/Variable/roomFacilities.pkl', 'wb') as f:
    pickle.dump(roomFacilities, f)
with open('../Notebook/Variable/nearestPointOfInterests.pkl', 'wb') as f:
    pickle.dump(nearestPointOfInterests, f)

## Combine Dataframe

In [37]:
dfRoom.drop('Room_originalRate', axis= 1, inplace=True)
dfHotel.drop('Hotel_originalRate', axis= 1, inplace=True)
dfPoint.drop('Point_originalRate', axis= 1, inplace=True)
dfHotelEx.drop('originalRate', axis= 1, inplace=True)
dfRoomEx.drop('originalRate', axis= 1, inplace=True)
df = df[['originalRate']]

C:\Users\goobe\AppData\Local\Temp\ipykernel_26416\2147204924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRoomEx.drop('originalRate', axis= 1, inplace=True)


In [38]:
combine = [df,  dfRoomEx, dfHotelEx, dfRoom, dfHotel, dfPoint]
df = pd.concat(combine, axis=1)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [39]:
# Check if there is any duplicate column
for i in range (0, len(df.columns)):
   for j in range (i+1, len(df.columns)):
      if df.columns[i] == df.columns[j]:
         print(df.columns[i])

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4871 entries, 0 to 4870
Columns: 199 entries, originalRate to Point_SCHOOL
dtypes: float64(169), int64(9), uint8(21)
memory usage: 6.7 MB


In [41]:
df.to_csv('../Dataset/encoded_data.csv', index=False)

In [42]:
col = df.columns
print(col)
# export col to another notebook
with open('../Notebook/Variable/col.pkl', 'wb') as f:
    pickle.dump(col, f)

Index(['originalRate', 'size', 'baseOccupancy', 'maxChildOccupancy',
       'maxChildAge', 'isBreakfastIncluded', 'isWifiIncluded', 'isRefundable',
       'hasLivingRoom', 'starRating',
       ...
       'Point_MUSEUM', 'Point_GALLERY', 'Point_PLACE_OF_WORSHIP',
       'Point_TRAIN_STATION', 'Point_ZOO', 'Point_ENTERTAINMENT',
       'Point_GARDEN', 'Point_THEATER', 'Point_STORE', 'Point_SCHOOL'],
      dtype='object', length=199)
